# Psi4 Notebook for educational purposes


This notebook is used to compare the results of Psi4 with the results with my own implementation of the Hartree-Fock method. It helps me to check if my implementation is correct. 

Connect to pc with `Psi4` installation and host jupyter server.

`ssh -L 8080:localhost:8080 username@pcname`

`jupyter notebook --no-browser --port=8080`

In [133]:
ls

eri_Psi4_test.dat  h2o_test.py*    h2o_testV2.py*        psi4_setup_h2.py
h2o_test.log       h2o_testV2.log  ipython_h2o_setup.py  timer.dat
h2o_test.out       h2o_testV2.out  output.dat


My setup, that is run:

In [134]:
cat psi4_setup_h2.py

import psi4
import numpy as np 

np.set_printoptions(precision=5, linewidth=100, suppress=True)
psi4.set_memory("4 GB")
psi4.core.set_output_file('output.dat', False)
numpy_memory = 4

mol = psi4.geometry("""
0 1
H 0.0 0.0 0.0
H 0.0 0.0 1.4
units bohr
""")

psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "sto-3g", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8})

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]



In [135]:
%run psi4_setup_h2.py

### This is with STO-3G basis set 

In [136]:
V

array([[-1.88044, -1.19483],
       [-1.19483, -1.88044]])

In [137]:
S

array([[1.     , 0.65932],
       [0.65932, 1.     ]])

### This is with 6-311G basis set 

In [138]:
psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "6-311G", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "MAXITER": 250, "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8, "CFOUR_CONTRACTION": "SEGMENTED", "SCF__PRINT_BASIS": "true"})

In [139]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V_ne = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

In [140]:
S

array([[1.     , 0.70638, 0.37366, 0.26948, 0.42526, 0.31026],
       [0.70638, 1.     , 0.78896, 0.42526, 0.72664, 0.67697],
       [0.37366, 0.78896, 1.     , 0.31026, 0.67697, 0.90422],
       [0.26948, 0.42526, 0.31026, 1.     , 0.70638, 0.37366],
       [0.42526, 0.72664, 0.67697, 0.70638, 1.     , 0.78896],
       [0.31026, 0.67697, 0.90422, 0.37366, 0.78896, 1.     ]])

In [141]:
V_ne

array([[-2.82405, -1.55802, -0.77545, -0.7038 , -0.97076, -0.65609],
       [-1.55802, -1.54664, -1.03651, -0.97076, -1.19542, -0.92183],
       [-0.77545, -1.03651, -0.96188, -0.65609, -0.92183, -0.89488],
       [-0.7038 , -0.97076, -0.65609, -2.82405, -1.55802, -0.77545],
       [-0.97076, -1.19542, -0.92183, -1.55802, -1.54664, -1.03651],
       [-0.65609, -0.92183, -0.89488, -0.77545, -1.03651, -0.96188]])